# Tutorial 2 : Linear regression and trees

# Tutorial 3 : First level of explicabity: statistics

In this class we will use data from the same dataset we used in the previous TD. We will build upon the work done previously.
* We will be using python for the data analysis (R is accepted as well if needed)
* The main libraries we are going to use are pandas,SHAP, matplotlib and sklearn
* This TD will not be graded.


Question 1.
Create a new notebook and from TD2 load the variable selection and the different models that you
created



In [43]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd

# fetch dataset 
heart_disease = fetch_ucirepo(id=45) 
  
# data (as pandas dataframes) 
X = heart_disease.data.features 
y = heart_disease.data.targets 

data =pd.concat([X.reset_index(drop=True), y], axis=1)

In [44]:
# variable information 
pd.DataFrame(heart_disease.variables)

,name,role,type,demographic,description,units,missing_values
0,age,Feature,Integer,Age,None,years,no
1,sex,Feature,Categorical,Sex,None,None,no
2,cp,Feature,Categorical,None,None,None,no
3,trestbps,Feature,Integer,None,resting blood pressure (on admission to the ho...,mm Hg,no
4,chol,Feature,Integer,None,serum cholestoral,mg/dl,no
5,fbs,Feature,Categorical,None,fasting blood sugar > 120 mg/dl,None,no
6,restecg,Feature,Categorical,None,None,None,no
7,thalach,Feature,Integer,None,maximum heart rate achieved,None,no
8,exang,Feature,Categorical,None,exercise induced angina,None,no
9,oldpeak,Feature,Integer,None,ST depression induced by exercise relative to ...,None,no


Important variables : 
* exang
* ca
* oldpeak

In [45]:
data.dropna(inplace=True)

In [46]:
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0


In [47]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# Convert from categorical to numeric data
from sklearn.preprocessing import OneHotEncoder


categorical_variables = heart_disease.variables[heart_disease.variables["type"] == "Categorical"].name
# Separate categorical and non-categorical data
categorical_data = data[categorical_variables]
non_categorical_data = data.drop(columns=categorical_variables)

enc = OneHotEncoder(sparse_output=False).fit(data)
encoded_data = enc.fit_transform(categorical_data)
# Convert encoded data to a DataFrame with appropriate column names
encoded_categorical_df = pd.DataFrame(encoded_data, columns=enc.get_feature_names_out(categorical_variables))

# Concatenate the encoded categorical data with the non-categorical data
data = pd.concat([non_categorical_data.reset_index(drop=True), encoded_categorical_df.reset_index(drop=True)], axis=1)

# Display the first few rows
data.head()

,age,trestbps,chol,thalach,oldpeak,ca,num,sex_0,sex_1,cp_1,...,restecg_1,restecg_2,exang_0,exang_1,slope_1,slope_2,slope_3,thal_3.0,thal_6.0,thal_7.0
0,63,145,233,150,2.3,0.0,0,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,67,160,286,108,1.5,3.0,2,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
2,67,120,229,129,2.6,2.0,1,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
3,37,130,250,187,3.5,0.0,0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,41,130,204,172,1.4,0.0,0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [48]:
target = (data['num'] > 1).astype(int)
del(data['num'])
data.head()

,age,trestbps,chol,thalach,oldpeak,ca,sex_0,sex_1,cp_1,cp_2,...,restecg_1,restecg_2,exang_0,exang_1,slope_1,slope_2,slope_3,thal_3.0,thal_6.0,thal_7.0
0,63,145,233,150,2.3,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,67,160,286,108,1.5,3.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
2,67,120,229,129,2.6,2.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
3,37,130,250,187,3.5,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,41,130,204,172,1.4,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.30)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, recall_score, f1_score
from sklearn.model_selection import GridSearchCV

rf_clf = RandomForestClassifier()
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_features': [7, 8, 9, 10],
    'n_estimators': [100, 200, 300, 1000]
}

# GridSearch to find the best parameters
grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=0)

# Fit the model with GridSearch
grid_search.fit(X_train, y_train)

# Best parameters and best estimator
print("Best Parameters:", grid_search.best_params_)
best_rf_model = grid_search.best_estimator_

# Test the performance with the tuned hyperparameters
y_pred_tuned = best_rf_model.predict(X_test)

rf_acc = accuracy_score(y_pred_tuned,y_test)
rf_F1 = f1_score(y_pred_tuned,y_test)
rf_rec=recall_score(y_pred_tuned,y_test)


Best Parameters: {'bootstrap': True, 'max_features': 7, 'n_estimators': 200}


Question 2.
Install the shap package (documentation here https://shap.readthedocs.io/en/latest/index.html)

In [50]:
%pip install shap

935.99s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



Question 3.
Let’s begin with an enrichment on the random forest interpretation.
* Get the variable importance of your features
* Recall the explanation you gave on them
* using the Treeexplainer function of the shap package, find the shapley values
* Visualise a couple of the explanations that are given with shap.forceplot. Can you explain them?
* Using the ”summary plot” with the ”plot type” equal to bar feature can you compare the importance
that shapley gives to the importance that the native ”variable importance” has given you?
* Do you understand the plot if you remove the plot type?


In [51]:
import shap

# Calculate SHAP values
rf_explainer = shap.Explainer(best_rf_model, X_train)
rf_shap_values = rf_explainer.shap_values(X_train)

# Plot variable importance
shap.summary_plot(rf_shap_values, X_train)

ImportError: Numba needs NumPy 2.0 or less. Got NumPy 2.1.


Question 4.
Now lets work on a different model. Let’s train an Xgboost
How does Xgboost work ? what is boosting? you can search the internet for these answers
* Install the Xgboost library if it’s not already installed
* Fit an Xgboost to your data and fine tune it
* Is it better than your Random Forest? why?



Question 5.
Let’s compare the feature importance of the Random Forest and the Xgboost
* Using shap’s tree explainer get the shapley values for this new model.
* Select the variables that are the most important and plot a dependency plot. Does the result confirm
your intuition?
* Compare the summary plot to the summary plot of the random forest. What are the changes?



Question 6.
We can use shapley values from the Xgboost to learn more about the properties. Let’s cluster the
properties based on the shapley values.
* Let’s reduce the dimension of our data for visualisation purposes. Do a PCA with the shapley values
and visualise the 2 principal axis.
* Given the visualisation choose a clustering algorithm (K-Means, DBSCAN, gausian mixture...) and try to cluster the shapley values of the property.
* Visualise your results
* What characteristics can you give to each cluster?
* What conclusion can you reach?